In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [14]:
# Get images
image = img_to_array(load_img('man.jpg'))
image = np.array(image, dtype=float)

In [26]:
image[0]

array([[  46.,   38.,   62.],
       [  46.,   39.,   57.],
       [  32.,   26.,   38.],
       ..., 
       [  88.,   82.,  110.],
       [  92.,   86.,  114.],
       [  87.,   81.,  109.]])

In [29]:
#rgb2lab(1.0/255*image)[0]

In [16]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [17]:
X.shape, Y.shape

((1, 400, 400, 1), (1, 400, 400, 2))

In [18]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [19]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')

In [20]:
model.fit(x=X, y=Y, batch_size=1, epochs=10)

Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 0.2238
Epoch 2/10
1/1 [==============================] - 1s 988ms/step - loss: 0.7935
Epoch 3/10
1/1 [==============================] - 1s 834ms/step - loss: 0.3514
Epoch 4/10
1/1 [==============================] - 1s 711ms/step - loss: 0.0422
Epoch 5/10
1/1 [==============================] - 1s 743ms/step - loss: 0.0198
Epoch 6/10
1/1 [==============================] - 1s 646ms/step - loss: 0.0193
Epoch 7/10
1/1 [==============================] - 1s 631ms/step - loss: 0.0191
Epoch 8/10
1/1 [==============================] - 1s 674ms/step - loss: 0.0188
Epoch 9/10
1/1 [==============================] - 1s 705ms/step - loss: 0.0186
Epoch 10/10
1/1 [==============================] - 1s 626ms/step - loss: 0.0183


In [21]:
print(model.metrics_names)
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

['loss']
1/1 [==============================] - 0s 442ms/step
0.0181250479072


C:\Users\shubham saksham\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\shubham saksham\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
